In [13]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [26]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras import layers
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,BatchNormalization,Dropout,AveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense, GaussianNoise, GlobalMaxPooling2D
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import cv2

In [15]:
#Downloading dataset
!kaggle datasets download -d vuppalaadithyasairam/bone-fracture-detection-using-xrays

bone-fracture-detection-using-xrays.zip: Skipping, found more recently modified local copy (use --force to force download)


In [16]:
import zipfile
zip_ref = zipfile.ZipFile('/content/bone-fracture-detection-using-xrays.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [17]:
#Loading the images
x_train= keras.utils.image_dataset_from_directory(
    directory = '/content/archive (6)/train',
    labels='inferred',
    label_mode = 'int',
    batch_size=100,
    image_size=(256,256)
)

x_test= keras.utils.image_dataset_from_directory(
    directory = '/content/archive (6)/val',
    labels='inferred',
    label_mode = 'int',
    batch_size=100,
    image_size=(256,256)
)

Found 8863 files belonging to 2 classes.
Found 600 files belonging to 2 classes.


In [18]:
class_names = x_train.class_names
class_names

['fractured', 'not fractured']

In [21]:
# Normalizing
def process(image,label):
    image = tf.cast(image/255. ,tf.float32)
    return image,label

x_train = x_train.map(process)
x_test = x_test.map(process)

In [22]:
input_shape = (256, 256, 3)

In [36]:
#model
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=input_shape))
model.add(GaussianNoise(0.1))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(AveragePooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(GaussianNoise(0.1))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(GaussianNoise(0.1))
model.add(AveragePooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [37]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 256, 256, 32)      896       
                                                                 
 gaussian_noise_3 (Gaussian  (None, 256, 256, 32)      0         
 Noise)                                                          
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 128, 128, 32)      0         
 g2D)                                                            
                                                                 
 dropout_5 (Dropout)         (None, 128, 128, 32)      0         
                                                                 
 conv2d_6 (Conv2D)           (None, 128, 128, 64)      18496     
                                                                 
 average_pooling2d_2 (Avera  (None, 64, 64, 64)       

In [38]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [39]:
history = model.fit(x_train,epochs=5, batch_size=100, validation_data=x_test)

Epoch 1/5
89/89 [==============================] - 2573s 29s/step - loss: 0.8401 - accuracy: 0.4941 - val_loss: 0.6926 - val_accuracy: 0.6000
Epoch 2/5
89/89 [==============================] - 2571s 29s/step - loss: 0.6932 - accuracy: 0.5008 - val_loss: 0.6923 - val_accuracy: 0.6000
Epoch 3/5
89/89 [==============================] - 2566s 29s/step - loss: 0.6932 - accuracy: 0.5033 - val_loss: 0.6914 - val_accuracy: 0.6000
Epoch 4/5
89/89 [==============================] - 2599s 29s/step - loss: 0.6935 - accuracy: 0.5038 - val_loss: 0.6920 - val_accuracy: 0.6000
Epoch 5/5
89/89 [==============================] - 2552s 29s/step - loss: 0.6933 - accuracy: 0.5027 - val_loss: 0.6913 - val_accuracy: 0.6000
